<a href="https://colab.research.google.com/github/nearbykatayama/my_first_project/blob/main/GooRecipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#instalando o SDK do Google
!pip install -q -U google-generativeai

In [11]:
#configurações iniciais
import google.generativeai as genai
import os
import requests
import mimetypes
import tempfile

In [12]:
from google.colab import userdata
api_key= userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

In [13]:
#listando os modelos disponíveis
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [14]:
#configurações complementares
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
}

safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

In [15]:
#instrução
system_instruction = "Você é um mestre cuca, e conhece todos os pratos típicos do Brasil, Itália, França e Japão."

In [16]:
#modelo utilizado
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

In [ ]:
#código do chatbot - GooRecipe
chat = model.start_chat(history=[])

while True:
    tipo_imagem = input("O que você tem em mente? (quero fazer com o que tenho / me inspire): ")

    if tipo_imagem.lower() not in ["quero fazer com o que tenho", "me inspire"]:
        print("Opção inválida. Por favor, digite 'quero fazer com o que tenho' ou 'me inspire'.")
        continue

    imagem = input("Digite o caminho da imagem desejada: ")

    if tipo_imagem.lower() == "quero fazer com o que tenho":
        prompt = [genai.upload_file(imagem), "Diga qual prato é o da imagem, faça uma lista dos ingredientes necessários e as quantidades, e detalhe o modo de preparo. Dê opções como forno, microondas e outros. Dê uma estimativa de quanto tempo o prato fica pronto."]
    else:
        prompt = [genai.upload_file(imagem), "Sugira 3 opções de pratos que podem ser feitos com estes ingredientes, coloque as quantidades necessárias dos ingredientes, e detalhe o modo de preparo. Dê opções como forno, microondas e outros. Dê uma estimativa de quanto tempo o prato ficará pronto."]

    response = chat.send_message(prompt)
    print(response.text)  # acessar o texto gerado diretamente

    continuar = input("Quer tentar outro prato? (s/n): ")
    if continuar.lower() != "s":
        break
    chat = model.start_chat(history=[])  # reiniciar o chat para o próximo prato


O que você tem em mente? (quero fazer / me inspire): me inspire
Digite o caminho da imagem desejada: /content/ingredientes1.jpg
## Opções de Pratos com os Ingredientes: 

Com os ingredientes da imagem, temos uma variedade de pratos possíveis devido à versatilidade dos mesmos. Aqui estão três sugestões com diferentes modos de preparo:

**1. Pizza de Frigideira (20 minutos):**

* **Ingredientes:**
    * 1 base de pizza pronta (ou massa de pizza caseira) 
    * 2 colheres de sopa de molho de tomate
    * 1/2 xícara de queijo parmesão ralado
    * 1 tomate fatiado
    * 1/2 xícara de cogumelos fatiados
    * Orégano e manjericão a gosto
    * Azeite de oliva
* **Modo de Preparo:**
    1. Aqueça um fio de azeite em uma frigideira grande.
    2. Coloque a base da pizza e deixe dourar levemente de um lado.
    3. Vire a base, espalhe o molho de tomate, o queijo, o tomate e os cogumelos. 
    4. Tempere com orégano, manjericão e tampe a frigideira.
    5. Deixe cozinhar em fogo baixo por cerca

In [ ]:
#codigo de impressão
import textwrap

def format_chat_history(chat_history):
    """Formata o histórico do chat para exibição."""
    for message in chat_history:
        role = message.role.upper()
        print(f"**{role}**:")
        for part in message.parts:
            text = textwrap.indent(part.text, '  ')
            print(text)
        print('-------------------------------------------')